# ACETONE tutorial #1

**Generating the code from a given network**

In this notebook, we generate the C code corresponding to a **Acas** neural network, described in two formats: *ONNX* and *NNet*. We then use a random dataset to infere our code and check that the values remain consistent.

In the first part of the notebook, we instantiate the main class of ACETONE and use it to generate code.

In the second part, we compile the generated code and run it, before comparing the several outputs given by the package.

We will show that ACETONE remains consistent regardless of the format of the input.

* When running this notebook on Colab, we need to install ACETONE 
* If you run this notebook locally, run it in the environment in which you installed ACETONE

In [ ]:
#TODO check install on collab

In [ ]:
# Cleaning the working environment
from pathlib import Path
from os import remove, listdir, mkdir

# Path to the example files
PATH_DIR = Path("../tests/models/acas/acas_COC")

# Path to generated directories
nnet_output_path = Path("demo_acas_nnet")
onnx_output_path = Path("demo_acas_onnx")

files_directories = [onnx_output_path, nnet_output_path]

for directory in files_directories:
    if directory.exists():
        for file in listdir(directory):
            if not (directory / file).is_dir():
                remove(directory / file)

if not Path("./schedmcore_ACAS_CaseStudy/simulation_traces/").exists():
    mkdir(Path("./schedmcore_ACAS_CaseStudy/simulation_traces/"))

## Imports

In [ ]:
import numpy as np
import numpy.random as rd

import onnx 
import onnxruntime as rt

from acetone_nnet import CodeGenerator
from acetone_nnet import cli_compare

from schedmcore_ACAS_CaseStudy.src.main import run_simulation

## Generating code

There is two way to generate the code:
* Using the function 'cli_acetone' to directly generate both the output python and the code C
* Using the class 'CodeGenerator' to have more controle on the generation

The first method is mainly used as a command-line, either by runing the python file, either by using the built in command: *acetone_generate*.
Confere to the ReadMe for example using a terminal.
The second method is prefered when using the package. 
It allows more regarding the type of the arguments, give more controle over the generation.


The network we'll use as an exemple here is an ACAS with 6 Dense layers, each separated by a Relu function.

![acas](data/acas.png)

We'll consider both the [*ONNX*](../tests/models/acas/acas_COC/nn_acas_COC.onnx) and [*NNet*](../tests/models/acas/acas_COC/nn_acas_COC.nnet) format of this model. 

### Instantiating a **CodeGenerator** element

The essential parameter for a **CodeGenerator** element is *model_path*, the path to the model of interest. Some optional parameters can also be given to personalize the generated code:

* *test_dataset* : The set of input we will use to test the generated code (must be of shape __(nb_tests , input_shape)__)
* *function_name* : The name of the generated function
* *nb_tests* : The number of tests we want to run
* *normalize* : A boolean indicating if a normalization operator must be applied (only used for the *NNet* format)
* *versions* : A dictionary specifying the implementation for a layer (confer [tutorial #2](./tutorial2_using_variants.ipynb))
* *debug_mode* : A string indicating the type of model we want to debug (confer [tutorial #3](./tutorial3_using_debug_mode.ipynb))


In this introduction, we only consider the first three optional arguments.

The output path argument is later used to specify where the computed output must be stored.

In [ ]:
model_path = PATH_DIR / "nn_acas_COC.nnet"

test_dataset = rd.default_rng(10).random((1,5), dtype=np.float32)
function_name = "demo_acas"
nb_tests = 1

In [ ]:
# Create an ACETONE CodeGenerator from the model
nnet_generator = CodeGenerator(file=model_path,
                            function_name=function_name,
                            test_dataset=test_dataset,
                            nb_tests=nb_tests)

### Generating the C code

We use the *generate_c_file* methode to generate the code. 

In [ ]:
nnet_generator.generate_c_files(nnet_output_path)

By looking into the file explorer, we can now see that a few files have been generated in the *demo_acas_nnet* directory (which was created if it did not already exist):

* [*global_vars.c*](./demo_acas_nnet/global_vars.c)  : Initialization of model parameters

* [*inference.h*](./demo_acas_nnet/inference.h)    : Header declaration of the model parameters and the inference function
* [*inference.c*](./demo_acas_nnet/inference.c)    : Definition of the inference function
* [*test_dataset.h*](./demo_acas_nnet/test_dataset.h) : Declaration of global prameters (input size, number of test, ...) and of the test inputs
* [*test_dataset.c*](./demo_acas_nnet/test_dataset.c) : Initialization of the test inputs
* [*main.c*](./demo_acas_nnet/main.c)         : Main function, calls the inference on the input and write the result in a file
* [*Makefile*](./demo_acas_nnet/Makefile)       : Makefile to compile the C code

The neural network himself if contained in the first three files, while the later 4 provides an example of usage.

### Importing the ONNX model

Instead of a path to the saved file, we can also directly use the model (imported or created using both Keras and ONNX's native Python librairies) as an  input to ACETONE's **CodeGenerator**.

In [ ]:
model_path = PATH_DIR / "nn_acas_COC.onnx"
onnx_model = onnx.load(model_path)

# Create an ACETONE CodeGenerator from the ONNX model
onnx_generator = CodeGenerator(file=onnx_model,
                                function_name=function_name,
                                test_dataset=test_dataset,
                                nb_tests=nb_tests)

onnx_generator.generate_c_files(onnx_output_path)

## Generating the Python output

Now that we have our code, we use the *compute_inference* methode to compute a first evaluation of the inference function on the inputs, using ACETONE's python implementation of the layers. This computation method is used as a reference for the user, to check that the implemented C code returns consistent values.


In [ ]:
# Computing the inference for the nnet model
nnet_output = nnet_generator.compute_inference(nnet_output_path)
print(nnet_output)

In [ ]:
# Computing the inference for the onnx model
onnx_output = onnx_generator.compute_inference(onnx_output_path)
print(onnx_output)

## Compiling and running the generated code

Once the code has been generated, and the first inference has been done, the only remaining step is to compile and run the C code. And that's what the Makefile's `all` command is there for !

In [ ]:
# Compiling the files
! make -C demo_acas_nnet all

<div class="alert alert-block alert-info">
⚠️ When running the executable file, do not forget to add as parameter the path to the text file in which the ouptut will be written.
</div>


In [ ]:
# Running the executable
! ./demo_acas_nnet/demo_acas ./demo_acas_nnet/output_c.txt

Similary, we compile and run the code from the onnx model.

In [ ]:
! make -C demo_acas_onnx all

In [ ]:
! ./demo_acas_onnx/demo_acas ./demo_acas_onnx/output_c.txt

## Comparing two ouptuts

To verify if the two code did give the same value, we use the function *cli_compare*.

This command takes as input the path to two ouptut files (C or python) and the number of test done (here 1), and compare them term to term, returning the maximum absolute and relative errors.

In [ ]:
# Comparing the C and python ouptuts computing from the NNet format
cli_compare((nnet_output_path / "output_python.txt"), (nnet_output_path / "output_c.txt"), 1)

In [ ]:
# Comparing the C and python ouptuts computing from the ONNX format
cli_compare((onnx_output_path / "output_python.txt"), (onnx_output_path / "output_c.txt"), 1)

In [ ]:
# Comparing both C ouptuts
cli_compare((onnx_output_path / "output_c.txt"), (nnet_output_path / "output_c.txt"), 1)

Even though the generated code himself will change to fit the original network (example: in *ONNX*, the Dense layer is not implemented, thus a combination of a MatMul and an Add are used as a substitute), the output is the same for both networks, demonstrating the robustness of the framework to the input format.

The small error between python and c outputs being around `1e-08`, it is considered to be numerical. The values being stored as `float32` is C, and `float64` in Python support that theory.  

## Comparing with *ONNX*

We can also use *ONNX*'s official inference package, *onnxruntime*, to get an external reference and validate our models.



In [ ]:
model_path = PATH_DIR / "nn_acas_COC.onnx"

# Inferring the model
sess = rt.InferenceSession(model_path)
input_name = sess.get_inputs()[0].name
result = sess.run(None, {input_name: test_dataset[0]})
onnx_result = result[0].ravel().flatten()



max_error = 0.0
max_rel_error = 0.0
for i in range(5):
    diff = abs(onnx_output[i] - onnx_result[i])
    norm = abs(onnx_output[i]) + abs(onnx_result[i])
    max_error = max(max_error , diff)
    if norm != 0:
        max_rel_error = max(max_rel_error, diff/(norm/2))

print("Result given by onnxruntime :",onnx_result)
print("Result given by ACETONE :",onnx_output)
print("Maximal absolute error between them : ",max_error)
print("Maximal relative error between them : ",max_rel_error)

The comparison between *ONNX*'s official inference package and ACETONE's python output gives a similar result, with a maximal relative error around `1e-8`, showing our closeness to the reference.

## Study Case 

To be a bit more visual, and verify that the generated code preserves the semantics well, we use the simulator **schedmcore**, developped by Arthur Clavière, 
Laura Altieri Sambartolomé, Eric Asselin, Christophe Garion and Claire Pagetti in [*Verification of machine learning based cyber-physical systems: a comparative study*](https://dl.acm.org/doi/abs/10.1145/3501710.3519540).

This simulator is located in the directory [schedmcore_ACAS_CaseStudy](./schedmcore_ACAS_CaseStudy/README.md).

We start with a simple simulation, with two aircraft travelling in the same horizontal plane: the *ownship*, equipped with an ACASXU based controller and the *intruder*. 

In [ ]:
# Simulation using the native implementation
run_simulation(
    system_name="acasxu",
    path_initial_states="./schedmcore_ACAS_CaseStudy/init_states/init_states_acasxu.csv",
    directory_results="./schedmcore_ACAS_CaseStudy/simulation_traces/",
    mode="schedmcore",
    )

In [ ]:
# Simulation using the native implementation
run_simulation(
    system_name="acasxu",
    path_initial_states="./schedmcore_ACAS_CaseStudy/init_states/init_states_acasxu.csv",
    directory_results="./schedmcore_ACAS_CaseStudy/simulation_traces/",
    mode="acetone",
    )

In this second scenario, both aircraft have acces to a ACASXU based controller.

In [ ]:
run_simulation(
    system_name="acasxu_2",
    path_initial_states="./schedmcore_ACAS_CaseStudy/init_states/init_states_acasxu_2.csv",
    directory_results="./schedmcore_ACAS_CaseStudy/simulation_traces/",
    mode="schedmcore",
    )

In [ ]:
run_simulation(
    system_name="acasxu_2",
    path_initial_states="./schedmcore_ACAS_CaseStudy/init_states/init_states_acasxu_2.csv",
    directory_results="./schedmcore_ACAS_CaseStudy/simulation_traces/",
    mode="acetone",
    )